<a href="https://colab.research.google.com/github/Watthana65022128/model-vit-vision-transformer/blob/main/ThaiHandWrite_Transformer_Based_OCR_tones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ขั้นตอนหลัก
1.การเตรียมข้อมูล:

แปลงภาพตัวอักษรภาษาไทยให้เป็นรูปแบบ Tensor ที่เหมาะสมสำหรับโมเดล Transformer
จัดแบ่งข้อมูลเป็นชุด Train, Validation, และ Test
การสร้างโมเดล OCR ด้วย Transformer:

ใช้โมเดล Vision Transformer (ViT) หรือ Swin Transformer จากไลบรารี เช่น Hugging Face Transformers
2.การฝึกโมเดล:

กำหนด Loss Function และ Optimizer เพื่อปรับปรุงโมเดลให้เหมาะสม
3.การประเมินผล:

ใช้เมตริก เช่น Accuracy, Precision, Recall และ F1-score

คำอธิบาย
การเตรียมข้อมูล:

ข้อมูลถูกแปลงเป็น Tensor ที่เหมาะสมสำหรับโมเดล Transformer โดยใช้ ViTFeatureExtractor
การสร้างโมเดล:

ใช้ Vision Transformer (google/vit-base-patch16-224) ที่ pretrained บน ImageNet และปรับแต่งให้รองรับจำนวนคลาสตัวอักษรไทย
การฝึกโมเดล:

ฝึกโมเดลด้วย Optimizer (AdamW) และ Loss Function (CrossEntropyLoss) พร้อมวัดผลบนชุด Validation
การประเมินผล:

ใช้ classification_report เพื่อวัดเมตริก เช่น Precision, Recall และ F1-score

# :0. อ่านข้อมูลภาพและสร้าง label จากชื่อโฟลเดอร์

# :1. การเตรียมข้อมูล

In [ ]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 1. ติดตั้ง nbstripout
!pip install nbstripout

# 2. ทำความสะอาดไฟล์
!nbstripout ThaiHandWrite_Transformer_Based_OCR_tones.ipynb


Could not strip 'ThaiHandWrite_Transformer_Based_OCR_tones.ipynb': file not found


In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class ThaiHandwrittenDataset(Dataset):
    def __init__(self, data_dirs, transform=None):
        self.data = []
        self.labels = []
        self.transform = transform

        # อ่านข้อมูลจากแต่ละโฟลเดอร์
        for label, folder in enumerate(data_dirs.keys()):
            folder_path = data_dirs[folder]
            if not os.path.exists(folder_path):
                print(f"Warning: Folder {folder_path} does not exist. Skipping.")
                continue

            print(f"Processing folder: {folder_path}")

            # ตรวจสอบว่าโฟลเดอร์มี subfolders หรือไม่
            subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
            if not subfolders:
                print(f"Warning: No subfolders found in {folder_path}. Skipping.")
                continue

            # วนลูปในแต่ละ subfolder (คลาส)
            for subfolder in subfolders:
                subfolder_path = os.path.join(folder_path, subfolder)
                print(f"Processing subfolder: {subfolder_path}")

                # ตรวจสอบไฟล์ใน subfolder
                folder_files = os.listdir(subfolder_path)
                print(f"Files in {subfolder_path}: {folder_files}")

                for img_name in folder_files:
                    img_path = os.path.join(subfolder_path, img_name)

                    # ข้ามไฟล์ที่ไม่ใช่ไฟล์จริง
                    if not os.path.isfile(img_path):
                        print(f"Skipping non-file entry: {img_path}")
                        continue

                    # กรองเฉพาะไฟล์ภาพที่มีนามสกุลที่รองรับ
                    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        print(f"Skipping non-image file: {img_name}")
                        continue

                    # ตรวจสอบว่าไฟล์เป็นภาพหรือไม่
                    try:
                        with Image.open(img_path) as img:
                            img.verify()  # ตรวจสอบว่าไฟล์สามารถเปิดได้
                            self.data.append(img_path)
                            self.labels.append(subfolder)  # ใช้ชื่อ subfolder เป็น label
                    except (IOError, SyntaxError) as e:
                        print(f"Skipping invalid image file: {img_path}, Error: {e}")

        # เพิ่มดีบักเมื่อไม่มีข้อมูล
        if len(self.data) == 0:
            print("Dataset is empty. Please check the data directories.")
            self.data = None
            self.labels = None

    def __len__(self):
        return len(self.data) if self.data else 0

    def __getitem__(self, idx):
        if not self.data:
            raise ValueError("Dataset is empty. Cannot access any items.")

        img_path = self.data[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

# การโหลด Dataset
data_dirs = {
    "tones": "/content/drive/MyDrive/characters_dataset/tones"
}

# กำหนด Transform สำหรับการแปลงภาพ
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

try:
    dataset = ThaiHandwrittenDataset(data_dirs=data_dirs, transform=transform)
    if not dataset.data:
        raise ValueError("Dataset is empty after processing. Please check the data directories.")
    print(f"Dataset loaded with {len(dataset)} samples.")
except ValueError as e:
    print(e)


Processing folder: /content/drive/MyDrive/characters_dataset/tones
Processing subfolder: /content/drive/MyDrive/characters_dataset/tones/โท02.png
Files in /content/drive/MyDrive/characters_dataset/tones/โท02.png: ['02-001.png', '02-035.png', '02-002.png', '02-068.png', '02-046.png', '02-024.png', '02-112.png', '02-090.png', '02-396.png', '02-013.png', '02-407.png', '02-334.png', '02-057.png', '02-223.png', '02-079.png', '02-418.png', '02-429.png', '02-385.png', '02-124.png', '02-279.png', '02-113.png', '02-301.png', '02-101.png', '02-146.png', '02-190.png', '02-212.png', '02-135.png', '02-246.png', '02-157.png', '02-157(1).png', '02-235.png', '02-290.png', '02-312.png', '02-168.png', '02-224.png', '02-146(1).png', '02-179.png', '02-201.png', '02-268.png', '02-257.png', '02-323.png', '02-135(1).png', '02-390.png', '02-386.png', '02-394.png', '02-335.png', '02-384.png', '02-397.png', '02-381.png', '02-380.png', '02-357.png', '02-387.png', '02-391.png', '02-383.png', '02-346.png', '02-368

In [ ]:
from PIL import Image
import os
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms

class ThaiHandwrittenDataset(Dataset):
    def __init__(self, data_dir=None, data=None, labels=None, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

        if not self.data or not self.labels:
            raise ValueError("Data and labels cannot be None.")

        if len(self.data) != len(self.labels):
            raise ValueError("Data and labels length mismatch.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label


# การโหลด Dataset
try:
    # กำหนด path ของโฟลเดอร์หลัก
    data_dir = '/content/drive/MyDrive/characters_dataset/tones'
    dataset = ThaiHandwrittenDataset(data_dir=data_dir, transform=None)
    if not dataset.data:  # ตรวจสอบว่า dataset มีข้อมูลหรือไม่
        raise ValueError("Dataset is empty after processing. Please check the data directory.")
    print(f"Dataset loaded with {len(dataset)} samples.")
except ValueError as e:
    print(e)

# หาก dataset ไม่ว่าง ให้ดำเนินการต่อ
if dataset and dataset.data:
    # แบ่งข้อมูลเป็น Train และ Test
    train_data, test_data, train_labels, test_labels = train_test_split(
        dataset.data, dataset.labels, test_size=0.2, random_state=42
    )

    # กำหนด Transform สำหรับการแปลงภาพ
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # สร้าง Dataset ใหม่ที่ใช้ Transform และกำหนด data และ labels
    train_dataset = ThaiHandwrittenDataset(data=train_data, labels=train_labels, transform=transform)
    test_dataset = ThaiHandwrittenDataset(data=test_data, labels=test_labels, transform=transform)

    # สร้าง DataLoader
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    print("Example batch from train_loader:")
    for imgs, labels in train_loader:
        print("Image paths:", imgs[:5])
        print("Labels:", labels[:5])
        break
else:
    print("Cannot proceed without a valid dataset.")



Data and labels cannot be None.
Example batch from train_loader:
Image paths: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 

# :2. การสร้าง Train model using Vision Transformers (ViT) และ Test

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import classification_report
import pandas as pd
from torchvision import transforms

# ThaiHandwrittenDataset (เหมือนเดิม)
class ThaiHandwrittenDataset(Dataset):
    def __init__(self, data=None, labels=None, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

        if not self.data or not self.labels:
            raise ValueError("Data and labels cannot be None.")

        if len(self.data) != len(self.labels):
            raise ValueError("Data and labels length mismatch.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

# การปรับขนาดภาพเป็น 224x224
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ปรับขนาดภาพเป็น 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # ใช้ค่า normalization ที่เหมาะสม
])

# การโหลดข้อมูล
data_dir = '/content/drive/MyDrive/characters_dataset/tones'

data = []
labels = []

if not os.path.exists(data_dir):
    raise ValueError(f"Data directory {data_dir} does not exist.")

for label, folder in enumerate(os.listdir(data_dir)):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)

            try:
                with Image.open(img_path) as img:
                    img.verify()  # ตรวจสอบว่าไฟล์ภาพถูกต้อง
                    data.append(img_path)
                    labels.append(label)  # ใช้ index ของ folder เป็น label
            except (IOError, SyntaxError):
                print(f"Skipping invalid image file: {img_path}")

# แบ่งข้อมูลเป็น Train และ Test
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

# สร้าง Dataset ใหม่ที่ใช้ Transform และกำหนด data และ labels
train_dataset = ThaiHandwrittenDataset(data=train_data, labels=train_labels, transform=transform)
test_dataset = ThaiHandwrittenDataset(data=test_data, labels=test_labels, transform=transform)

# สร้าง DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# โหลด ViT โมเดลจาก HuggingFace
vit_model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=len(set(train_labels)))

# กำหนด device (CPU/GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model = vit_model.to(device)

# กำหนด Loss function และ Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vit_model.parameters(), lr=1e-4)

# ตัวแปรสำหรับเก็บข้อมูล loss และ accuracy
epoch_losses = []
epoch_accuracies = []

# ฟังก์ชันสำหรับฝึกโมเดล
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []

        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            imgs = imgs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(imgs)  # ทำนายผล
            loss = criterion(outputs.logits, labels)  # คำนวณ loss
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        epoch_losses.append(epoch_loss)
        epoch_accuracies.append(epoch_accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}%")

        # Classification Report
        if epoch == num_epochs - 1:
            report = classification_report(all_labels, all_preds, output_dict=True)
            report_df = pd.DataFrame(report).transpose()
            report_df.to_excel(f"classification_report_epoch_{epoch+1}.xlsx")

# ฟังก์ชันสำหรับทดสอบโมเดล
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc="Testing"):
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)  # ทำนายผล
            _, predicted = torch.max(outputs.logits, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.4f}%")

    # Classification Report
    report = classification_report(all_labels, all_preds, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_df.to_excel(f"test_classification_report.xlsx")

# เริ่มการฝึกโมเดล
num_epochs = 5
train_model(vit_model, train_loader, criterion, optimizer, num_epochs=num_epochs)

# ทดสอบโมเดล
test_model(vit_model, test_loader)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/5: 100%|██████████| 45/45 [47:02<00:00, 62.72s/it]


Epoch [1/5], Loss: 0.4202, Accuracy: 90.1269%


Epoch 2/5:  89%|████████▉ | 40/45 [40:35<05:01, 60.39s/it]

In [ ]:
# บันทึกโมเดลหลังจากฝึก
torch.save(vit_model.state_dict(), 'vit_model_checkpoint_tones.pth')


In [ ]:
# โหลดโมเดลเพื่อฝึกต่อ
vit_model.load_state_dict(torch.load('vit_model_checkpoint_tones.pth'))

# :4. กราฟแสดงผลลัพธ์:

In [ ]:
# ฟังก์ชันสำหรับสร้างกราฟ
def plot_results():
    # Loss graph
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(epoch_losses) + 1), epoch_losses, label='Training Loss', color='blue')
    plt.title('Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.savefig("loss_per_epoch.png")
    plt.show()

    # Accuracy graph
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(epoch_accuracies) + 1), epoch_accuracies, label='Training Accuracy', color='green')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.grid(True)
    plt.savefig("accuracy_per_epoch.png")
    plt.show()

# สร้างกราฟการฝึก
plot_results()


# :5 Save Graph